<center><a target="_blank" href="https://githubtocolab.com/sayan1999/YouTube-Video-Summarizer/blob/main/summary.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a></center>

## Imports

In [14]:
import os
from yt_dlp import YoutubeDL
import glob
import webvtt
import pandas as pd

from scipy import spatial
from gensim.models import word2vec

from collections import namedtuple
import nltk
import pandas as pd
import gensim
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [15]:
def dl_transcript(url):
    with YoutubeDL({"skip_download":True, "writeautomaticsub":True, "subtitleslangs":["en", "en-us", "en-us", "en-uk", "en-in", "en-es", "en-fr"]}) as ydl:
        if ydl.download(url):
            print("-----------------------------------------------------------------------------------------------------------LINK FAILED")
            return None, None
        else:
            # print(ydl.extract_info(url))
            op = [f for f in os.listdir() if f.startswith(os.path.splitext(ydl.prepare_filename(ydl.extract_info(url)))[0]) and f.endswith('.vtt')]
            if op:
              return ydl.extract_info(url)['title'], op[0]
            else:
              print("-----------------------------------------------------------------------------------------------------------file download FAILED")
              return None, None

In [89]:
def init():
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    model = gensim.models.KeyedVectors.load_word2vec_format(
        "/home/instantinopaul/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz",
        binary=True,
    )
    # model = None
    return tokenizer, model


def docsimilarity(model, keyword, doc):
    cutoff = 0.4
    score = 0
    for w in doc:
        sm = model.similarity(keyword, w) if w in model else 0
        if sm >= cutoff:
            score += sm
    return score


def get_relevant_line(df, searchphrase, model):
    wordnet_lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words("english"))
    df = preprocess(df, wordnet_lemmatizer, stop_words)
    
    keywords = [ wordnet_lemmatizer.lemmatize(
            wordnet_lemmatizer.lemmatize(wordnet_lemmatizer.lemmatize(kw.lower()), pos="v"),
            pos=("a"),
        ) for kw in tokenizer.tokenize(searchphrase)]
    
    df["similarity"] = sum(
        [
            df["docs"].apply(lambda doc: docsimilarity(model, keyword.lower(), doc))
            for keyword in keywords
            if keyword in model
        ]
    )
    df["docs"] = df["docs"].apply(" ".join)
    df = df.sort_values("similarity", ascending=False)
    df.to_csv('result.csv', index=False)
    return df[df["similarity"]>1][['start', 'end']].values.tolist()

In [16]:
import re
def parse_subtitles(url):
  title, vttfile=dl_transcript(url)
  if vttfile is None:
    return "SOME ISSUE WITH VIDEO LINK OR DOWNLOAINDING VIDEO CONTENTS"
  print("text file expected", vttfile+'.txt')
  os.system(f"cat \"{vttfile}\" | grep : -v | awk '!seen[$0]++' > \"{vttfile}.txt\"")
  tscript = re.sub(r'[\s|\n]',' ',open(f'{vttfile}.txt').read().replace('WEBVTT', '', 1))
  data = [[caption.start, caption.end, caption.text] for caption in webvtt.read(vttfile)]
  df = pd.DataFrame(data, columns=['start', 'end', 'caps'])
  return title, tscript, df

In [85]:
def preprocess(df, wordnet_lemmatizer, stop_words):
    orig_docs = [
        [
            word        for word in tokenizer.tokenize(sent)
        ]
        for sent in df['caps']
    ]
    
    df['docs'] = [
        [
            wordnet_lemmatizer.lemmatize(
                wordnet_lemmatizer.lemmatize(wordnet_lemmatizer.lemmatize(word.lower()), pos="v"),
                pos=("a"),
            )
            for word in sent
            if word not in stop_words
        ]
        for sent in orig_docs
    ]
    print(df['docs'])
    return df

In [18]:
title, tscript, df = parse_subtitles('https://www.youtube.com/watch?v=6Ow2zHJQw2M')

[youtube] Extracting URL: https://www.youtube.com/watch?v=6Ow2zHJQw2M
[youtube] 6Ow2zHJQw2M: Downloading webpage
[youtube] 6Ow2zHJQw2M: Downloading ios player API JSON
[youtube] 6Ow2zHJQw2M: Downloading android player API JSON
[youtube] 6Ow2zHJQw2M: Downloading m3u8 information
[info] 6Ow2zHJQw2M: Downloading subtitles: en
[info] 6Ow2zHJQw2M: Downloading 1 format(s): 616+251
Deleting existing file Israel Continues To Bomb Palestinians - As the UN Watches Helplessly on Day 26 ｜ Akash Banerjee [6Ow2zHJQw2M].en.vtt
[info] Writing video subtitles to: Israel Continues To Bomb Palestinians - As the UN Watches Helplessly on Day 26 ｜ Akash Banerjee [6Ow2zHJQw2M].en.vtt
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 3
[download] Destination: Israel Continues To Bomb Palestinians - As the UN Watches Helplessly on Day 26 ｜ Akash Banerjee [6Ow2zHJQw2M].en.vtt
[download] 100% of   17.29KiB in 00:00:01 at 9.16KiB/s                 
[youtube] Extracting URL: https://www.youtube.co

In [22]:
tokenizer, model = init()

In [90]:
searchphrase = "children death"
get_relevant_line(df, searchphrase, model)

0      [this, jabalia, refugee, camp, israel, drop, 6...
1      [accord, report, explosive, power, bomb, equal...
2      [the, entire, refugee, camp, destroy, more, 40...
3      [and, still, many, child, many, innocent, pale...
4      [israel, even, try, deny, time, they, say, yes...
                             ...                        
127    [but, know, youtube, might, restrict, video, w...
128          [so, please, ignore, video, comment, share]
129    [and, tell, people, raise, voice, not, ignore,...
130    [it, important, disappoint, genocide, gaza, te...
131    [otherwise, today, gaza, tomorrow, another, pl...
Name: docs, Length: 132, dtype: object


[['00:10:35.477', '00:10:44.114'],
 ['00:04:30.424', '00:04:41.642'],
 ['00:08:08.720', '00:08:18.994'],
 ['00:08:51.922', '00:08:58.454']]